<a href="https://colab.research.google.com/github/sam80FDS/Sam/blob/main/AI_Shorts_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-Cost YouTube Shorts Factory - AI Rendering Server

This notebook runs on **Google Colab Free Tier** and provides a FastAPI server for rendering viral tech news shorts.

**Architecture:**
- Edge-TTS for voice generation (free, unrestricted)
- EchoMimic/SadTalker for avatar animation
- FFmpeg for audio mixing and compositing
- ngrok for public HTTPS tunnel to n8n
- Google Drive for storage

**Cost: $0 (100% free tier)**

## Section 1: Setup and Dependencies

Install all required packages for audio/video processing, API server, and Drive integration.

In [ ]:
import os
import subprocess
import sys

# Install system dependencies
print("Installing system dependencies...")
subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
subprocess.run(['apt-get', 'install', '-y', 'ffmpeg', 'libsm6', 'libxext6'],
               check=True, capture_output=True)

# Install Python packages
packages = [
    'fastapi',
    'uvicorn',
    'pyngrok',
    'nest_asyncio',
    'edge-tts',
    'opencv-python',
    'scipy',
    'requests',
    'python-multipart',
    'aiofiles'
]

print("Installing Python packages...")
for package in packages:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', package],
                   check=True, capture_output=True)

print("✓ All dependencies installed successfully!")

## Section 2: Google Drive Integration and Asset Paths

Mount Google Drive and validate the folder structure for music and output directories.

In [ ]:
from google.colab import drive
import os
from pathlib import Path

print("Mounting Google Drive...")
drive.mount('/content/drive')

# Define asset paths
DRIVE_BASE = '/content/drive/MyDrive'
MUSIC_FOLDER = os.path.join(DRIVE_BASE, 'Shorts_Assets/Music')
OUTPUT_FOLDER = os.path.join(DRIVE_BASE, 'Shorts_Output')
TEMP_FOLDER = '/content/temp_shorts'

# Create necessary directories
for folder in [MUSIC_FOLDER, OUTPUT_FOLDER, TEMP_FOLDER]:
    os.makedirs(folder, exist_ok=True)
    print(f"✓ Folder ready: {folder}")

# Validate music folder has content
music_files = [f for f in os.listdir(MUSIC_FOLDER) if f.endswith(('.mp3', '.wav', '.m4a'))]
print(f"\n✓ Found {len(music_files)} music tracks in Music folder")
if len(music_files) < 3:
    print("⚠️  WARNING: Less than 3 music tracks. Recommendation: Upload 20-30 safe tracks to ensure variety.")
    print("   See Section 9 for music sources.")

# Environment variables
os.environ['MUSIC_FOLDER'] = MUSIC_FOLDER
os.environ['OUTPUT_FOLDER'] = OUTPUT_FOLDER
os.environ['TEMP_FOLDER'] = TEMP_FOLDER
print("\n✓ Environment variables configured")

## Section 3: Edge-TTS Voice Generation

Generate speech audio from script text using Edge-TTS with viral-optimized voices.

In [ ]:
import asyncio
import edge_tts
from scipy.io import wavfile
import wave
import struct

async def generate_voice_audio(script_text: str, voice: str = "en-US-ChristopherNeural",
                               output_path: str = None) -> tuple:
    """
    Generate speech audio from script text using Edge-TTS.

    Args:
        script_text: The script to convert to speech
        voice: Voice ID (ChristopherNeural, GuyNeural, AriaNeural recommended)
        output_path: Where to save the audio file

    Returns:
        (audio_file_path, duration_seconds)
    """
    if output_path is None:
        output_path = os.path.join(os.environ['TEMP_FOLDER'], 'voice_output.mp3')

    # Create communicate object
    communicate = edge_tts.Communicate(script_text, voice, rate="+0%", volume="+0%")

    # Save audio file
    await communicate.save(output_path)

    # Get duration using ffprobe
    result = subprocess.run(
        ['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
         '-of', 'default=noprint_wrappers=1:nokey=1:nokey=1', output_path],
        capture_output=True, text=True
    )

    duration = float(result.stdout.strip())

    print(f"✓ Voice generated: {duration:.2f} seconds")
    return output_path, duration

# Test voice generation
async def test_voice():
    test_script = "This changes everything. Artificial intelligence just got upgraded. Your tools are about to feel obsolete."
    voice_file, duration = await generate_voice_audio(test_script)
    print(f"✓ Test voice file: {voice_file}")
    print(f"✓ Duration: {duration:.2f}s")
    return voice_file, duration

# Run test
asyncio.run(test_voice())

## Section 4: FFmpeg Audio Mixing with Auto-Ducking

Mix voice audio with background music using FFmpeg, with automatic volume ducking to prevent masking.

In [ ]:
import random

def select_random_music(music_folder: str = None) -> str:
    """Randomly select one music file from the Music folder."""
    if music_folder is None:
        music_folder = os.environ['MUSIC_FOLDER']

    music_files = [f for f in os.listdir(music_folder)
                   if f.endswith(('.mp3', '.wav', '.m4a', '.aac'))]

    if not music_files:
        raise FileNotFoundError(f"No music files found in {music_folder}")

    selected = random.choice(music_files)
    full_path = os.path.join(music_folder, selected)
    print(f"✓ Selected music: {selected}")
    return full_path

def mix_audio_with_auto_ducking(voice_file: str, voice_duration: float,
                                 output_audio: str = None,
                                 music_volume: float = 0.1) -> str:
    """
    Mix voice audio with background music using FFmpeg.
    Auto-ducking: Music volume is reduced to 10% so voice is always clear.

    Args:
        voice_file: Path to voice MP3/WAV
        voice_duration: Duration of voice in seconds
        output_audio: Output audio file path
        music_volume: Volume level for music (0.0-1.0, default 0.1 = 10%)

    Returns:
        Path to mixed audio file
    """
    if output_audio is None:
        output_audio = os.path.join(os.environ['TEMP_FOLDER'], 'mixed_audio.wav')

    # Select random music
    music_file = select_random_music()

    # Create FFmpeg command with auto-ducking
    ffmpeg_cmd = [
        'ffmpeg', '-y',
        '-i', voice_file,
        '-i', music_file,
        '-filter_complex',
        f'[1]volume={music_volume}[mus];[0][mus]amix=inputs=2:duration=first[out]',
        '-map', '[out]',
        '-t', str(voice_duration),  # Trim to voice duration
        '-c:a', 'pcm_s16le',  # WAV codec
        '-ar', '44100',  # 44.1kHz sample rate
        output_audio
    ]

    print(f"Mixing audio (voice + {music_volume*100:.0f}% music)...")
    result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"FFmpeg Error: {result.stderr}")
        raise RuntimeError("FFmpeg audio mixing failed")

    print(f"✓ Audio mixed: {output_audio}")
    return output_audio

# Test audio mixing
voice_file, voice_duration = asyncio.run(test_voice())
mixed_audio = mix_audio_with_auto_ducking(voice_file, voice_duration)
print(f"✓ Test complete: {mixed_audio}")

## Section 5: Avatar Animation with EchoMimic/SadTalker

Generate animated avatar video from audio using EchoMimic (recommended for free tier).

**Note:** For zero-cost, we use a simplified approach with a static avatar + subtle movement animation. For production quality, integrate EchoMimic properly after cloning the repo.

In [ ]:
import cv2
import numpy as np

def create_placeholder_animated_avatar(duration: float, width: int = 512, height: int = 512,
                                       output_path: str = None) -> str:
    """
    Create a placeholder animated avatar video.

    PRODUCTION NOTE: Replace this with actual EchoMimic integration:
    1. Clone: git clone https://github.com/BadToBest/EchoMimic.git
    2. Use EchoMimic to animate real avatar with audio
    3. This placeholder generates subtle animations for testing

    Args:
        duration: Video duration in seconds
        width, height: Video dimensions
        output_path: Output video path

    Returns:
        Path to avatar video file
    """
    if output_path is None:
        output_path = os.path.join(os.environ['TEMP_FOLDER'], 'avatar.mp4')

    # Video properties
    fps = 30
    total_frames = int(duration * fps)

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print(f"Creating placeholder avatar video: {total_frames} frames at {fps} fps...")

    # Create frames with subtle animation
    for frame_idx in range(total_frames):
        # Create a frame with gradient background (simulating avatar face)
        frame = np.zeros((height, width, 3), dtype=np.uint8)

        # Gradient background (dark blue to cyan)
        for y in range(height):
            frame[y, :] = [int(100 + y * 0.05), int(150 + y * 0.02), 200]

        # Draw animated circle (simulating breathing/blinking)
        center_x, center_y = width // 2, height // 2
        radius = 80 + int(10 * np.sin(frame_idx * 0.1))  # Breathing animation

        cv2.circle(frame, (center_x, center_y), radius, (255, 200, 100), -1)

        # Add subtle eye animation
        eye_y_offset = int(5 * np.sin(frame_idx * 0.15))
        cv2.circle(frame, (center_x - 25, center_y - 20 + eye_y_offset), 8, (0, 0, 0), -1)
        cv2.circle(frame, (center_x + 25, center_y - 20 + eye_y_offset), 8, (0, 0, 0), -1)

        # Write frame
        out.write(frame)

        if (frame_idx + 1) % (fps * 10) == 0:
            print(f"  Generated {frame_idx + 1}/{total_frames} frames...")

    out.release()
    print(f"✓ Avatar video created: {output_path}")
    return output_path

# Test avatar generation
test_avatar = create_placeholder_animated_avatar(5.0)
print(f"✓ Test avatar: {test_avatar}")

## Section 6: Video Compositing and Rendering

Compose the animated avatar onto a background image and generate the final video with audio.

In [ ]:
import urllib.request
from PIL import Image

def download_and_prepare_background(image_url: str, width: int = 1080, height: int = 1920) -> str:
    """
    Download background image from URL and resize for 9:16 vertical format.

    Args:
        image_url: URL of background image
        width: Output width (default 1080 for YouTube Shorts)
        height: Output height (default 1920 for 9:16 aspect ratio)

    Returns:
        Path to resized background image
    """
    bg_path = os.path.join(os.environ['TEMP_FOLDER'], 'background.jpg')

    print(f"Downloading background image from {image_url}...")
    urllib.request.urlretrieve(image_url, bg_path)

    # Open and resize
    img = Image.open(bg_path)
    img = img.resize((width, height), Image.Resampling.LANCZOS)
    img.save(bg_path)

    print(f"✓ Background prepared: {width}x{height}")
    return bg_path

def composite_avatar_on_background(avatar_video: str, background_image: str,
                                    audio_file: str, output_video: str = None,
                                    avatar_scale: float = 0.5, avatar_x_pos: float = 0.5) -> str:
    """
    Composite animated avatar onto background and add audio.

    Args:
        avatar_video: Path to avatar animation video
        background_image: Path to background image
        audio_file: Path to mixed audio (voice + music)
        output_video: Output video path
        avatar_scale: Scale of avatar relative to frame (0-1)
        avatar_x_pos: Horizontal position (0=left, 0.5=center, 1=right)

    Returns:
        Path to final composited video
    """
    if output_video is None:
        output_video = os.path.join(os.environ['TEMP_FOLDER'], 'final_composite.mp4')

    print("Compositing avatar onto background...")

    # FFmpeg filter: Overlay avatar at 50% size, centered
    filter_complex = (
        f"[0]scale=540:960[avatar];"  # Scale avatar to 50% (1080x1920 -> 540x960)
        f"[1][avatar]overlay=(main_w-overlay_w)/2:(main_h-overlay_h)/2[bg];"  # Center overlay
        f"[bg]pad=width=1080:height=1920:x=0:y=0[final]"  # Ensure 1080x1920
    )

    ffmpeg_cmd = [
        'ffmpeg', '-y',
        '-loop', '1', '-i', background_image,
        '-i', avatar_video,
        '-i', audio_file,
        '-filter_complex', filter_complex,
        '-map', '[final]',
        '-map', '2:a',
        '-c:v', 'libx264',
        '-preset', 'fast',  # Fast for Colab
        '-c:a', 'aac',
        '-shortest',  # Stop when audio ends
        output_video
    ]

    result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"FFmpeg Error: {result.stderr}")
        raise RuntimeError("FFmpeg compositing failed")

    # Get video size
    size_result = subprocess.run(
        ['ffprobe', '-v', 'error', '-show_entries', 'stream=width,height',
         '-of', 'csv=p=0', output_video],
        capture_output=True, text=True
    )
    print(f"✓ Final video created: {output_video}")
    print(f"  Resolution: {size_result.stdout.strip()}")

    return output_video

# Note: To test compositing, we'd need a background image URL
# Example: background = download_and_prepare_background("https://example.com/news-bg.jpg")

## Section 7: FastAPI Endpoint for /render-short

Create the FastAPI server with the `/render-short` endpoint for orchestrating the entire rendering pipeline.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import json
from datetime import datetime
import nest_asyncio

# Enable nested asyncio for Jupyter
nest_asyncio.apply()

# Initialize FastAPI
app = FastAPI(title="YouTube Shorts Rendering Server", version="1.0.0")

# Request model
class RenderRequest(BaseModel):
    script_text: str
    background_image_url: str
    voice: str = "en-US-ChristopherNeural"  # Default viral voice
    project_id: str = None  # Optional identifier for tracking

@app.get("/health")
async def health_check():
    """Health check endpoint for n8n webhook monitoring."""
    return {
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "service": "YouTube Shorts Rendering Server"
    }

@app.post("/render-short")
async def render_short(request: RenderRequest):
    """
    Main rendering endpoint.

    Orchestrates: Voice Generation -> Avatar Animation -> Audio Mixing -> Compositing

    Args:
        request: RenderRequest with script_text and background_image_url

    Returns:
        {
            "status": "success",
            "video_path": "path/to/final/video.mp4",
            "duration_seconds": 50,
            "project_id": "xyz123"
        }
    """
    try:
        print(f"\n{'='*60}")
        print(f"Starting render: {request.project_id or 'unnamed'}")
        print(f"Script length: {len(request.script_text)} chars")
        print(f"Voice: {request.voice}")
        print(f"{'='*60}")

        # Step 1: Generate voice audio
        print("\n[STEP 1/4] Generating voice audio...")
        voice_file, voice_duration = await generate_voice_audio(
            request.script_text,
            voice=request.voice
        )

        # Step 2: Create avatar animation
        print("\n[STEP 2/4] Creating avatar animation...")
        avatar_video = create_placeholder_animated_avatar(voice_duration)
        # FUTURE: Replace with EchoMimic integration

        # Step 3: Mix audio (voice + music with auto-ducking)
        print("\n[STEP 3/4] Mixing audio (voice + background music at 10%)...")
        mixed_audio = mix_audio_with_auto_ducking(voice_file, voice_duration)

        # Step 4a: Download and prepare background
        print("\n[STEP 4a/4] Downloading background image...")
        background_image = download_and_prepare_background(request.background_image_url)

        # Step 4b: Composite and render final video
        print("\n[STEP 4b/4] Compositing final video...")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        final_video = os.path.join(
            os.environ['OUTPUT_FOLDER'],
            f"short_{request.project_id or timestamp}.mp4"
        )

        final_video = composite_avatar_on_background(
            avatar_video,
            background_image,
            mixed_audio,
            final_video
        )

        print(f"\n{'='*60}")
        print(f"✓ RENDER COMPLETE")
        print(f"  Video: {final_video}")
        print(f"  Duration: {voice_duration:.2f}s")
        print(f"{'='*60}\n")

        return {
            "status": "success",
            "video_path": final_video,
            "duration_seconds": voice_duration,
            "project_id": request.project_id or timestamp,
            "timestamp": datetime.now().isoformat()
        }

    except Exception as e:
        print(f"ERROR: {str(e)}")
        raise HTTPException(
            status_code=500,
            detail=f"Rendering failed: {str(e)}"
        )

print("✓ FastAPI app configured with /health and /render-short endpoints")

## Section 8: ngrok Tunneling for n8n Webhooks

Launch the FastAPI server with ngrok to create a public HTTPS URL for n8n HTTP requests.

In [ ]:
import uvicorn
from pyngrok import ngrok
import threading
import time

# Configure ngrok (free tier)
# ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")  # Optional: For persistent URLs

def run_fastapi_server():
    """Run FastAPI server in a background thread."""
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

def start_server_with_ngrok(port: int = 8000, ngrok_auth_token: str = None) -> str:
    """
    Start FastAPI server and create ngrok tunnel.

    Args:
        port: Port to run FastAPI on
        ngrok_auth_token: Optional ngrok auth token for persistent URLs

    Returns:
        Public HTTPS URL for n8n to call
    """
    print("\n" + "="*60)
    print("Starting YouTube Shorts Rendering Server")
    print("="*60)

    # Set auth token if provided
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)

    # Start FastAPI in background thread
    server_thread = threading.Thread(target=run_fastapi_server, daemon=True)
    server_thread.start()

    print(f"✓ FastAPI server starting on port {port}...")
    time.sleep(2)  # Wait for server to start

    # Start ngrok tunnel
    try:
        public_url = ngrok.connect(port, "http")
        print(f"\n✓ ngrok tunnel established!")
        print(f"\n  PUBLIC NGROK URL: {public_url}")
        print(f"\n  Use this URL in n8n HTTP Request nodes:")
        print(f"  POST {public_url}/render-short")
        print(f"  POST {public_url}/health")
        print("\n" + "="*60)

        return str(public_url)

    except Exception as e:
        print(f"ERROR: Failed to create ngrok tunnel: {e}")
        print("Make sure ngrok is installed: pip install pyngrok")
        raise

# Launch the server
try:
    # Option 1: Without authentication (free tier, temporary URL)
    public_url = start_server_with_ngrok()

    # Save URL for reference
    with open(os.path.join(os.environ['TEMP_FOLDER'], 'server_url.txt'), 'w') as f:
        f.write(f"ngrok URL: {public_url}\n")
        f.write(f"Health Check: {public_url}/health\n")
        f.write(f"Render Endpoint: {public_url}/render-short\n")

    print("\n✓ Server URL saved to temp folder")
    print("\nServer is now running and ready for n8n requests!")
    print("Keep this notebook cell running to maintain the ngrok tunnel.")

except Exception as e:
    print(f"Failed to start server: {e}")
    print("Troubleshooting: Make sure all dependencies are installed")

## Section 9: Music Selection and Curation Strategy

Complete guide for sourcing copyright-free music for monetization-safe viral shorts.

# Zero-Cost Music Strategy for Viral YouTube Shorts

## Why This Matters
- **YouTube Monetization:** Any copyrighted music triggers content ID claims (you lose revenue)
- **Viral Edge:** Copyright-free ≠ boring. Modern royalty-free has incredible cinematic tracks
- **Zero Maintenance:** Upload 20-30 tracks ONCE to Drive, automate forever

---

## Recommended Free Music Sources

### 1. **YouTube Audio Library** (Best Option)
- **Cost:** FREE (YouTube account required)
- **Quality:** 10,000+ tracks, all monetization-safe
- **How to Download:**
  1. Go to YouTube Studio → Create → Audio Library
  2. Search by mood: "Inspirational", "Upbeat", "Cinematic"
  3. Download MP3 to your computer
  4. Upload to `/content/drive/MyDrive/Shorts_Assets/Music`
- **Best Tracks for Tech News:**
  - "Ukulele Happy" (upbeat, energetic)
  - "Bright Lights" (modern, tech-forward)
  - "Inspiring Cinematic" (authority, credibility)

### 2. **Free Music Archive (freemusicarchive.org)**
- **Cost:** FREE
- **License:** Creative Commons (verify attribution if needed)
- **Search Tags:** `upbeat`, `tech`, `electronic`, `energetic`
- **Download Tip:** Filter by "Shortest Duration" for short-form content (30-60s)

### 3. **Epidemic Sound (Free Tier)**
- **Cost:** Limited free tier (10 downloads/month)
- **Quality:** Studio-grade production
- **How to Use:** Download sparingly for critical moments

### 4. **CreatorMix (Indian Alternative)**
- **Cost:** FREE
- **Quality:** Curated Indian royalty-free music
- **Best For:** Diverse, energetic tech content
- **Website:** creatorix.in (or search "creatorix free music")

---

## Implementation: The "Mono-Music" Strategy

Instead of complex APIs, do this ONCE:

```
1. Download 25 tracks from YouTube Audio Library
2. Name them: track_001.mp3 → track_025.mp3
3. Upload to Google Drive: /MyDrive/Shorts_Assets/Music/
4. The system randomly selects one per short
5. Run automation forever (0 additional setup needed)
```

### Benefits
- ✅ No API keys, no monthly bills
- ✅ No copyright strikes
- ✅ 100% monetization-safe
- ✅ Variety maintained (25 tracks = 25 different shorts sound unique)
- ✅ Works forever (music doesn't change, API doesn't break)

---

## Verification Checklist Before Upload

Before uploading to Drive, verify each track:

```
✓ License: "Royalty-Free" or "Creative Commons 0" (public domain)
✗ AVOID: "Creative Commons Attribution" (requires crediting)
✗ AVOID: "Creative Commons Share-Alike"

✓ Duration: 30-90 seconds (short clips)
✓ Genre: Upbeat, energetic, modern
✓ BPM: 90-140 (pairs well with viral pacing)
```

---

## File Organization in Drive

```
MyDrive/
├── Shorts_Assets/
│   ├── Music/
│   │   ├── track_001.mp3
│   │   ├── track_002.mp3
│   │   ├── ...
│   │   └── track_025.mp3
│   └── (Other assets folder)
└── Shorts_Output/
    ├── short_20260122_001.mp4
    ├── short_20260122_002.mp4
    └── ...
```

---

## Cost Breakdown

| Item | Cost | Frequency |
|------|------|-----------|
| YouTube Audio Library | $0 | One-time setup (1 hour) |
| Google Drive Storage | $0 | Free 15GB included |
| Google Colab | $0 | Free tier unlimited |
| Edge-TTS (voice) | $0 | Microsoft free forever |
| FFmpeg | $0 | Open source |
| n8n (self-hosted) | $0 | Open source |
| **TOTAL MONTHLY COST** | **$0** | **$0** |

*Note: If you exceed Google Drive storage (15GB), upgrade to $1.99/100GB*

---

## Checklist: Setup in 30 Minutes

- [ ] Visit YouTube Audio Library
- [ ] Download 25 copyright-free tracks
- [ ] Create folder: `MyDrive/Shorts_Assets/Music`
- [ ] Upload all 25 tracks to that folder
- [ ] Run script below to verify folder integrity
- [ ] Done! System will randomize music forever


In [ ]:
# Music Folder Verification Script

def verify_music_folder(music_folder: str = None) -> dict:
    """
    Verify and report on music folder contents.
    Run this after uploading tracks to ensure setup is correct.
    """
    if music_folder is None:
        music_folder = os.environ['MUSIC_FOLDER']

    print("\n" + "="*60)
    print("MUSIC FOLDER VERIFICATION")
    print("="*60)

    # Check folder exists
    if not os.path.exists(music_folder):
        print(f"✗ ERROR: Folder not found: {music_folder}")
        return {"status": "failed"}

    print(f"✓ Folder found: {music_folder}\n")

    # List all audio files
    audio_extensions = ('.mp3', '.wav', '.m4a', '.aac', '.flac')
    music_files = [f for f in os.listdir(music_folder)
                   if f.lower().endswith(audio_extensions)]

    print(f"Found {len(music_files)} audio tracks:\n")

    total_size = 0
    for i, filename in enumerate(sorted(music_files), 1):
        filepath = os.path.join(music_folder, filename)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        total_size += size_mb
        print(f"  {i:2d}. {filename:40s} ({size_mb:6.2f} MB)")

    print(f"\n{'='*60}")
    print(f"Total Size: {total_size:.2f} MB")
    print(f"Recommendation: Aim for 20-30 tracks (100-500 MB)")

    if len(music_files) < 5:
        print("⚠️  WARNING: Less than 5 tracks. Add more for variety!")
    elif len(music_files) >= 20:
        print("✓ Excellent: Enough variety for many shorts")

    print(f"{'='*60}\n")

    return {
        "status": "success",
        "track_count": len(music_files),
        "total_size_mb": total_size,
        "tracks": music_files
    }

# Run verification
music_status = verify_music_folder()

# Print recommendations
if music_status["track_count"] < 5:
    print("\n📝 NEXT STEP: Upload at least 5 music tracks")
    print("   1. Go to YouTube Studio > Audio Library")
    print("   2. Download 5-25 royalty-free tracks")
    print("   3. Upload to:", os.environ['MUSIC_FOLDER'])
else:
    print("\n✓ Your music library is ready!")
    print("   System will randomly select tracks for each short")